In [ ]:
# setup to see the execution time in each cell

!pip install ipython-autotime
!pip install directory_structure
# !pip install wandb
%load_ext autotime

In [ ]:
import pandas as pd
import os
import glob
import PIL
from PIL import Image
import numpy as np
import cupy as cp
import matplotlib.pyplot as plt
import cv2
import cuml

In [ ]:
DRIVER_ROOT_DIR = "/kaggle/input/" # HC Directory
# DRIVER_ROOT_DIR = "/content/drive/MyDrive/DTSC 870/Code" #MT Directory

# no augmentation
# DATASET_02_TRAIN = DRIVER_ROOT_DIR + "fer2013/02_FER/train"
# DATASET_02_TEST = DRIVER_ROOT_DIR + "fer2013/02_FER/test"

# augmentation v.1
# DATASET_02_TRAIN = DRIVER_ROOT_DIR + "fer2013-aug/Aug_train"
# DATASET_02_TEST = DRIVER_ROOT_DIR + "fer2013-aug-test/Aug_test"

# augmentation v.2
DATASET_02_TRAIN = DRIVER_ROOT_DIR + "fer2013-aug-train-2/Aug_train_2"
DATASET_02_TEST = DRIVER_ROOT_DIR + "fer2013-aug-test-2/Aug_test_2"

In [ ]:
def get_total_train_num(dir):
    return len(glob.glob(dir))


def get_data_df(dir):
    # modified code from: https://www.kaggle.com/namgalielei/simple-load-images-and-count-number-of-each-class

    train_df = pd.DataFrame()

    trainset = glob.glob(dir)

    train_df['file'] = [img.split("/")[-1] for img in trainset]
    train_df['class'] = [img.split("/")[-2] for img in trainset]

    return train_df


def generate_set(df, dir, classes_):

    # new_df = pd.DataFrame()
    pixels = []
    class_ = []

    # trainset = glob.glob(dir)
    for i in range(len(df.index)):
        # get the absolute img path
        # e.g., Brain_tumor_images/<train or test>/<class label>/<file name>
        path = dir + "/" + df.iloc[i]["class"] + "/" +df.iloc[i]["file"]
        # print(img)
        img = Image.open(path)
        # print("Img: {} \tClass: {}".format(np.array(img).flatten(), df.iloc[i]["class"]))
        pixels.append(cp.asnumpy(cp.array(img)).flatten())
        # pixels.append(np.array(img))

        # y_true encoding here
        class_.append(classes_.index(df.iloc[i]["class"]))

        # end loop here

    # return train_df
    return pixels, class_

In [ ]:
fer_df_train = get_data_df(DATASET_02_TRAIN+"/*/*.jpg")
fer_df_test = get_data_df(DATASET_02_TEST+"/*/*.jpg")

classes = fer_df_train["class"].unique().tolist()

In [ ]:
x_train, y_train = generate_set(fer_df_train, DATASET_02_TRAIN, classes)
x_test, y_test = generate_set(fer_df_test, DATASET_02_TEST, classes)

In [ ]:
x_train_df = pd.DataFrame()
x_train_df['class'] = y_train

In [ ]:
fer_df_test["class"].hist()

In [ ]:
np.unique(x_train_df["class"].tolist())

In [ ]:
# from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from skimage.feature import local_binary_pattern
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from matplotlib.pyplot import figure
from sklearn.metrics import balanced_accuracy_score, accuracy_score
from numpy import mean

# from sklearn.cluster import MiniBatchKMeans, KMeans
from cuml.cluster import KMeans
from cuml.svm import SVC

In [ ]:
def generate_PCA_set(comp, x_train, x_test):
    pca = PCA(n_components=comp)
    __x_train__ = pca.fit_transform(x_train)
    __x_test__ = pca.transform(x_test)
    return __x_train__, __x_test__


def get_model():
#   return MySVC()
    return SVC()


def feature_scale(_x_train_, _x_test_):
    sc = StandardScaler()
    
    train_sc = sc.fit_transform(_x_train_)
    test_sc = sc.transform(_x_test_)
    
    return train_sc, test_sc


def feature_scale2(_x_train_):
    sc = StandardScaler()
    
    train_sc2 = sc.fit_transform(_x_train_)
    
    return train_sc2

In [ ]:
def generate_set2(df, dir, classes_):
    class_ = list()
    d = list()
    
    sift = cv2.SIFT_create()
    
    for i in range(len(df.index)):
        path = dir + "/" + df.iloc[i]["class"] + "/" +df.iloc[i]["file"]
        img = cv2.imread(path)
        keypoints_, descriptors_ = sift.detectAndCompute(img, None)
        
        # if there's no feature deceted, ignore this img
        if descriptors_ is None or len(descriptors_) == 0:
            continue
            
        d.append(descriptors_)
        class_.append(classes_.index(df.iloc[i]["class"]))
        # end loop here
        
    return d, class_

The Elbow method to find the optimal K for the K means.

However, this is not doable without the BoW histogram because not all images have the same number of features detected by the SIFT.

In [ ]:
def elbow(x_train_, k_list):
    wcss = list()
    
    for i in k_list:
        # intial with K-Means++ to avoid initalization trap
#         kmeans = MiniBatchKMeans(n_clusters=i, init="k-means++", random_state=42)
        kmeans = KMeans(n_clusters=i, init="scalable-k-means++", random_state=42)
        
        # train the matrix features
        kmeans.fit(x_train_)
        
        # get the WCSS val
        wcss.append(kmeans.inertia_)
    
    return wcss

In [ ]:
# x_train_sift, y_train_sift = generate_set2(fer_df_train, DATASET_02_TRAIN, classes)
# x_test_sift, y_test_sift = generate_set2(fer_df_test, DATASET_02_TEST, classes)

# x_train = [i.flatten() for i in x_train_sift]
# y_train = [i.flatten() for i in y_train_sift]
# x_test = [i.flatten() for i in x_test_sift]
# y_test = [i.flatten() for i in y_test_sift]

In [ ]:
# x_train

In [ ]:
# K_vals = [2, 7, 50, 100,150,200,350,400,450,500,530,550,600]
# WCSS = elbow(x_train, K_vals)

# plt.plot(range(1, 11), WCSS)
# plt.title("The Elbow Method: Number of Cluters")
# plt.xlabel("Number of clusters")
# plt.ylabel("WCSS")
# plt.show()

In [ ]:
def generate_set3(df: pd.DataFrame, dir_: str, classes_: list, K: int):
    features_ = list()
    class_ = list()
    d = list()
    
    
    ##### SIFT #####
    sift = cv2.SIFT_create()
    
    for i in range(len(df.index)):
        path = dir_ + "/" + df.iloc[i]["class"] + "/" +df.iloc[i]["file"]
        img = cv2.imread(path)
        keypoints_, descriptors_ = sift.detectAndCompute(img, None)
        
        if descriptors_ is None or len(descriptors_) == 0:
            continue
            
        d.append(descriptors_)
        class_.append(classes_.index(df.iloc[i]["class"]))
        # end loop
    
    
    ##### BoW Histogram #####
    # building cluster model
    mbKmean = KMeans(n_clusters=K, init="scalable-k-means++", random_state=42)
    # n_clusters = cluster_model.n_clusters
    des_stack = [desc for img in d for desc in img]
    all_descriptors = np.array(des_stack)
    mbKmean.fit(all_descriptors)
    
    
    # Calculting histogram vectors
    for dec in d:
        words = mbKmean.predict(dec)
        img_bow_hist = np.array([np.bincount(words, minlength=K)])
        features_.append(img_bow_hist)
        # end loop
    
    features_ = np.array(features_)
    
    return features_, class_

In [ ]:
# x_train_sift, y_train_sift = generate_set3(fer_df_train, DATASET_02_TRAIN, classes, 80)
# x_test_sift, y_test_sift = generate_set3(fer_df_test, DATASET_02_TEST, classes, 80)

# x_train_sift = np.asarray(x_train_sift, dtype=np.float32)
# y_train_sift = np.asarray(y_train_sift, dtype=np.float32)
# x_test_sift = np.asarray(x_test_sift, dtype=np.float32)
# y_test_sift = np.asarray(y_test_sift, dtype=np.float32)

In [ ]:
# shape_list = list()

# for item in x_train_sift:
#     shape_list.append(item.shape)

# print("The size of set after SIFT: {}".format(set(shape_list)))

In [ ]:
from sklearn.multiclass import OneVsRestClassifier

In [ ]:
def run_svm(x_train_sift, y_train_sift, x_test_sift, y_test_sift):
    
    # scaling
#     x_train_sc, x_test_sc = feature_scale(x_train_sift, x_test_sift)

#     x_train_sc, x_test_sc = np.array(x_train_sc), np.array(x_test_sc)
#     y_train = np.array(y_train_sift, dtype=np.float32)
    
    model = OneVsRestClassifier(SVC(random_state=1, multiclass_strategy="ovr", kernel="poly", degree=4))

    model.fit(x_train_sift, y_train_sift)

    # predict the train set
    y_trainHat = model.predict(x_train_sift)

    # predict the test set
    y_testHat = model.predict(x_test_sift)

    # compute the unweighted accuracy
    uw_acc = balanced_accuracy_score(y_test_sift, y_testHat)
    print("The unweighted accuracy: {}".format(uw_acc))

    # compute the weighted accuracy
    w_acc = accuracy_score(y_test_sift, y_testHat)
    print("The weighted accuracy: {}".format(w_acc))
    
    return model, uw_acc, w_acc, y_testHat

In [ ]:
N = list()
# K_vals = [7, 50, 100,150,200,350,400,450,500,530,550,600]
MAX_K = 60

# for k in K_vals:
for k in range(2, MAX_K):
    x_train_sift, y_train_sift = generate_set3(fer_df_train, DATASET_02_TRAIN, classes, k)
    x_test_sift, y_test_sift = generate_set3(fer_df_test, DATASET_02_TEST, classes, k)
    N.append({
        "k": k,
        "x_train": np.asarray(x_train_sift, dtype=np.float32),
        "y_train": np.asarray(y_train_sift, dtype=np.float32),
        "x_test": np.asarray(x_test_sift, dtype=np.float32),
        "y_test": np.asarray(y_test_sift, dtype=np.float32)
    })

In [ ]:
for item in N:
    print("K: {}".format(item["k"]))
    run_svm(item["x_train"], item["y_train"], item["x_test"], item["y_test"])
    print("\n")

In [ ]:
# k = 14
# x_train_sift, y_train_sift = generate_set3(fer_df_train, DATASET_02_TRAIN, classes, k)
# x_test_sift, y_test_sift = generate_set3(fer_df_test, DATASET_02_TEST, classes, k)

# item = {
#     "k": k,
#     "x_train": np.asarray(x_train_sift, dtype=np.float32),
#     "y_train": np.asarray(y_train_sift, dtype=np.float32),
#     "x_test": np.asarray(x_test_sift, dtype=np.float32),
#     "y_test": np.asarray(y_test_sift, dtype=np.float32)
# }

In [ ]:
# model, uw_acc, w_acc, y_pred2 = run_svm(item["x_train"], item["y_train"], item["x_test"], item["y_test"])

In [ ]:
# x_axis = list(range(2, 59))

In [ ]:
# import plotly.graph_objects as go

# fig = go.Figure()
# fig.add_trace(go.Line(
#     x=x_axis,
#     y=uw_acc,
#     mode='lines',
#     name='Unweighted Test Accuracy',
#     marker=dict(
#         color='red',
#         size=10
#     ))
# )

# fig.add_trace(go.Line(
#     x=x_axis,
#     y=w_acc,
#     mode='lines',
#     name='Weighted Test Accuracy',
#     marker=dict(
#         color='green',
#         size=10
#     ))
# )

# fig.update_layout(
#     title="SVM weighted VS unweighted accuracy with respect to K",
#     xaxis_title="K value",
#     yaxis_title="Accuracy (%)"
# )
# fig.update_traces(mode='markers+lines')

# fig.show()

In [ ]:
# from sklearn.metrics import plot_confusion_matrix
# from sklearn.metrics import classification_report
# from cuml.metrics import confusion_matrix

# import plotly.express as px

# from sklearn.metrics import roc_curve, auc
# from matplotlib.pyplot import figure

# from sklearn.preprocessing import LabelBinarizer

In [ ]:
# np.unique(y_pred2)

In [ ]:
# # scaling
# # x_train_sc, x_test_sc = feature_scale(item["x_train"], item["x_test"])

# # x_train_sc, x_test_sc = np.array(x_train_sc), np.array(x_test_sc)
# # y_train = np.array(y_train_sift, dtype=np.float32)

# y_pred2 = model.predict(item["x_test"])
# y_pred2 = np.array(y_pred2).astype(int)

# lb = LabelBinarizer()
# # temp = np.array(y_pred2[0])
# enc_y_pred2 = lb.fit_transform(y_pred2)
# enc_y_test_sift = lb.fit_transform(y_test_sift)

In [ ]:
# len(classes)

In [ ]:
# np.unique(y_test_sift)

In [ ]:
# import random

# enc_y_pred2[random.randint(3, len(enc_y_test_sift))]

In [ ]:
# # Compute ROC curve and ROC area for each class
# fpr = dict()
# tpr = dict()
# roc_auc = dict()
# for i in range(len(classes)):
#     fpr[i], tpr[i], _ = roc_curve(enc_y_test_sift[:,i], enc_y_pred2[:, i])
#     roc_auc[i] = auc(fpr[i], tpr[i])
    
# # Compute micro-average ROC curve and ROC area
# fpr["micro"], tpr["micro"], _ = roc_curve(enc_y_test_sift.ravel(), enc_y_pred2.ravel())
# roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

In [ ]:
# # Plot ROC curve
# plt.figure()
# plt.plot(fpr["micro"], tpr["micro"],
#          label='micro-average ROC curve (area = {0:0.2f})'
#                ''.format(roc_auc["micro"]))
# for i in range(len(classes)):
#     plt.plot(fpr[i], tpr[i], label='ROC curve of class {0} (area = {1:0.2f})'
#                                    ''.format(i, roc_auc[i]))

# # figure(figsize=(8, 6), dpi=80)
# plt.plot([0, 1], [0, 1], 'k--')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Some extension of Receiver operating characteristic to multi-class')
# plt.legend(bbox_to_anchor=(1.04,1), borderaxespad=0)
# plt.show()

In [ ]:
# y_pred_ = y_pred2
# print(classification_report(y_test_sift, y_pred_, target_names=classes))